In [17]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 23: LAN Party ---</h2><p>As The Historians wander around a secure area at Easter Bunny HQ, you come across posters for a <a href="https://en.wikipedia.org/wiki/LAN_party" target="_blank">LAN party</a> scheduled for today! Maybe you can find it; you connect to a nearby <a href="/2016/day/9">datalink port</a> and download a map of the local network (your puzzle input).</p>
<p>The network map provides a list of every <em>connection between two computers</em>. For example:</p>
<pre><code>kh-tc
qp-kh
de-cg
ka-co
yn-aq
qp-ub
cg-tb
vc-aq
tb-ka
wh-tc
yn-cg
kh-ub
ta-co
de-co
tc-td
tb-wq
wh-td
ta-ka
td-qp
aq-cg
wq-ub
ub-vc
de-ta
wq-aq
wq-vc
wh-yn
ka-de
kh-ta
co-tc
wh-qp
tb-vc
td-yn
</code></pre>
<p>Each line of text in the network map represents a single connection; the line <code>kh-tc</code> represents a connection between the computer named <code>kh</code> and the computer named <code>tc</code>. Connections aren't directional; <code>tc-kh</code> would mean exactly the same thing.</p>
<p>LAN parties typically involve multiplayer games, so maybe you can locate it by finding groups of connected computers. Start by looking for <em>sets of three computers</em> where each computer in the set is connected to the other two computers.</p>
<p>In this example, there are <code>12</code> such sets of three inter-connected computers:</p>
<pre><code>aq,cg,yn
aq,vc,wq
co,de,ka
co,de,ta
co,ka,ta
de,ka,ta
kh,qp,ub
qp,td,wh
tb,vc,wq
tc,td,wh
td,wh,yn
ub,vc,wq
</code></pre>
<p>If the Chief Historian is here, <em>and</em> he's at the LAN party, it would be best to know that right away. You're pretty sure his computer's name starts with <code>t</code>, so consider only sets of three computers where at least one computer's name starts with <code>t</code>. That narrows the list down to <code><em>7</em></code> sets of three inter-connected computers:</p>
<pre><code>co,de,<em>ta</em>
co,ka,<em>ta</em>
de,ka,<em>ta</em>
qp,<em>td</em>,wh
<em>tb</em>,vc,wq
<em>tc</em>,<em>td</em>,wh
<em>td</em>,wh,yn
</code></pre>
<p>Find all the sets of three inter-connected computers. <em>How many contain at least one computer with a name that starts with <code>t</code>?</em></p>
</article>


In [18]:
from collections import deque
from collections.abc import Sequence
from itertools import batched
from pprint import pprint
from typing import Self


tests = [
    {
        "name": "Example",
        "s": """
            kh-tc
            qp-kh
            de-cg
            ka-co
            yn-aq
            qp-ub
            cg-tb
            vc-aq
            tb-ka
            wh-tc
            yn-cg
            kh-ub
            ta-co
            de-co
            tc-td
            tb-wq
            wh-td
            ta-ka
            td-qp
            aq-cg
            wq-ub
            ub-vc
            de-ta
            wq-aq
            wq-vc
            wh-yn
            ka-de
            kh-ta
            co-tc
            wh-qp
            tb-vc
            td-yn
        """,
        "filter": lambda cs: True,
        "expected": 12,
    },
    {
        "name": "Example with filter",
        "s": """
            kh-tc
            qp-kh
            de-cg
            ka-co
            yn-aq
            qp-ub
            cg-tb
            vc-aq
            tb-ka
            wh-tc
            yn-cg
            kh-ub
            ta-co
            de-co
            tc-td
            tb-wq
            wh-td
            ta-ka
            td-qp
            aq-cg
            wq-ub
            ub-vc
            de-ta
            wq-aq
            wq-vc
            wh-yn
            ka-de
            kh-ta
            co-tc
            wh-qp
            tb-vc
            td-yn
        """,
        "filter": lambda cs: any(c.startswith("t") for c in cs),
        "expected": 7,
    },
]


class NetWork(Str):
    def __init__(self, s: str, filter: Callable[[Sequence], bool]) -> None:
        self.graph = self._parse(s)
        self.filter = filter

    def all_circles_of_size(self, exact_size: int = 3) -> int:
        queue = deque([[c] for c in sorted(self.graph.keys())])

        size = 1
        seen = set()

        while queue and size < exact_size:
            for _ in range(len(queue)):
                cycle = queue.popleft()

                queue.extend(
                    cycle + [c] for c in self.graph[cycle[-1]] if c > cycle[-1]
                )

            size += 1

        return sum(
            1
            for cycle in queue
            if cycle[0] in self.graph[cycle[-1]] and self.filter(cycle)
        )

    def _parse(self, s) -> defaultdict[str, set[str]]:
        graph = defaultdict(set[str])

        for c1, c2 in batched(re.findall(r"\w+", s), 2):
            graph[c1].add(c2)
            graph[c2].add(c1)

        return graph


@test(tests=tests[:])
def partI_test(s: str, filter: Callable[[Sequence], bool]) -> int:
    nw = NetWork(s, filter)
    res = nw.all_circles_of_size(3)
    return res


Test Example passed, for partI_test.
Test Example with filter passed, for partI_test.
Success


In [ ]:
with open("../input/day23.txt") as f:
    puzzle = f.read()

print(
    f"Part I: {NetWork(puzzle,lambda cs: any(c.startswith("t") for c in cs)).all_circles_of_size(3)}"
)

Part I: 1240


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1240</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><


<link href="style.css" rel="stylesheet"></link>


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>202972175280682</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
